In [1]:
# train and evaluate models of radiomics features
# please note that how VAE split the datasets


In [2]:
# This is how vae split the datasets
# # CV folds
# seed = 9001
# skf = StratifiedKFold(n_splits=fold,
#                         shuffle=True,
#                         random_state=seed)
# results = []
# for train_idx, test_idx in skf.split(X, Y):
#     if isinstance(X, pd.DataFrame):
#         X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#     elif isinstance(X, np.ndarray):
#         X_train, X_test = X[train_idx], X[test_idx]
#     if isinstance(Y, pd.DataFrame):
#         Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
#     elif isinstance(Y, np.ndarray):
#         Y_train, Y_test = Y[train_idx], Y[test_idx]
#     print("train", Counter(Y_train), "test", Counter(Y_test))


In [18]:
# labels
import os
import os.path as osp
import sys
import time
import numpy as np

sys.path.insert(1, "/labs/gevaertlab/users/yyhhli/code/vae")


In [4]:
# import the labels
from datasets.label.label_stanfordradiogenomics import LabelStfReGroup, \
    LabelStfTStage, LabelStfNStage, LabelStfAJCC, LabelStfHisGrade, \
        LabelStfRGLymphInvasion, LabelStfRGPleuralInvasion, LabelStfEGFRMutation, \
            LabelStfKRASMutation


/labs/gevaertlab/users/yyhhli/miniconda3/envs/lungvae38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# load the features
import pandas as pd
feature_df = pd.read_csv("/labs/gevaertlab/users/yyhhli/code/vae/baselines/pyradiomics/pyradiomics_features.csv")

In [6]:
feature_df

,Image,Mask,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,...,gradient_gldm_LargeDependenceLowGrayLevelEmphasis,gradient_gldm_LowGrayLevelEmphasis,gradient_gldm_SmallDependenceEmphasis,gradient_gldm_SmallDependenceHighGrayLevelEmphasis,gradient_gldm_SmallDependenceLowGrayLevelEmphasis,gradient_ngtdm_Busyness,gradient_ngtdm_Coarseness,gradient_ngtdm_Complexity,gradient_ngtdm_Contrast,gradient_ngtdm_Strength
0,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,v3.0.1,1.21.5,2.1.1.2,1.3.0,3.8.13,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [0.5, 1.0, 2...",afd6dd9c4d73dcc5984d14618cfd9bd24b39eec0,...,15.608196,0.140131,0.293587,45.697434,0.006642,1.167107,0.005220,543.979866,0.307530,1.181886
1,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,v3.0.1,1.21.5,2.1.1.2,1.3.0,3.8.13,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [0.5, 1.0, 2...",4bf171a43e350cc619df9690528b6d1ee88b4586,...,19.509045,0.191641,0.273686,43.661682,0.008080,2.819208,0.002872,539.276166,0.370494,0.821049
2,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,v3.0.1,1.21.5,2.1.1.2,1.3.0,3.8.13,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [0.5, 1.0, 2...",da5b3b871e31c29c8179b784fddab89af3591b91,...,8.322712,0.123496,0.357912,90.171486,0.011267,1.529061,0.003868,1216.135798,0.479142,1.846281
3,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,v3.0.1,1.21.5,2.1.1.2,1.3.0,3.8.13,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [0.5, 1.0, 2...",be1ef8acb488ce5ea1420f77b1a3f034c0bcece4,...,4.515643,0.088149,0.381907,99.805807,0.010795,0.930724,0.004367,1757.256841,0.440500,2.438068
4,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,v3.0.1,1.21.5,2.1.1.2,1.3.0,3.8.13,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [0.5, 1.0, 2...",403bb84a79db490953f94ae82deb711ced8fc0ab,...,48.640456,0.330551,0.202061,32.683886,0.008661,12.947123,0.000727,804.911775,0.162398,0.710956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,v3.0.1,1.21.5,2.1.1.2,1.3.0,3.8.13,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [0.5, 1.0, 2...",73ccc4915ad6998aea6609b59a222150237e4a7e,...,0.697978,0.035212,0.391649,95.192751,0.011390,3.594838,0.000815,2693.311009,0.285414,0.561963
139,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,v3.0.1,1.21.5,2.1.1.2,1.3.0,3.8.13,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [0.5, 1.0, 2...",5ebf081042cd2666e6e23e2f00f80edf37d13d2e,...,1.570383,0.057914,0.226596,27.927592,0.014326,12.238659,0.000281,1647.979694,0.080177,0.297700
140,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,v3.0.1,1.21.5,2.1.1.2,1.3.0,3.8.13,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [0.5, 1.0, 2...",c21a7ab268dec41446240b47a6034fa1ec8c1d0e,...,0.860598,0.047116,0.353948,63.117440,0.014701,2.624958,0.001160,2396.795969,0.188779,0.991932
141,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,/labs/gevaertlab/data/NSCLC_Radiogenomics/Resa...,v3.0.1,1.21.5,2.1.1.2,1.3.0,3.8.13,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [0.5, 1.0, 2.

In [7]:
# get the file names e.g. R01-001_img.nii.gz
# get numeric part as X

image_paths = feature_df["Image"].values

In [11]:
file_names = [osp.basename(path).split("_")[0] for path in image_paths]

In [12]:
# find columns that are numeric
numeric_cols = feature_df.select_dtypes(include=["int", "float"]).columns

In [16]:
# get them as X as a df
Xdf = feature_df[numeric_cols]

In [17]:
# convert to numpy array
X = Xdf.values

In [20]:
# to test we can try match label
t = LabelStfTStage()
labels = t.match_labels(file_names)

start matching 143 labels ...


100%|██████████| 143/143 [00:00<00:00, 163920.60it/s]

matching labels | 0.0 secs.


In [ ]:
# to test we can try define xgboost model

In [22]:
# base model
from functools import partial
import xgboost as xgb


RANDOM_DICT = {'random_state': 9001}
NJOB_DICT = {'n_jobs': -1}  # using all the processers

xgbtree = {'basemodel': partial(xgb.XGBClassifier, verbosity=1, **RANDOM_DICT, **NJOB_DICT),
           'params': dict(learning_rate=[0.01, 0.1],
                          n_estimators=[100],
                          reg_alpha=[1, 10])}

/labs/gevaertlab/users/yyhhli/miniconda3/envs/lungvae38/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [32]:
# deal with NAs
Y = np.array(labels)
na_idx = np.where(Y == "NA")[0]
# exclude them in both X and Y
X = np.delete(X, na_idx, axis=0)
Y = np.delete(Y, na_idx, axis=0)

In [34]:
# stratified split
from sklearn.model_selection import StratifiedKFold
from collections import Counter
seed = 9001
fold = 5
skf = StratifiedKFold(n_splits=fold,
                        shuffle=True,
                        random_state=seed)
for train_idx, test_idx in skf.split(X, Y):
    continue
x_train, x_test = X[train_idx], X[test_idx]
y_train, y_test = Y[train_idx], Y[test_idx]

In [35]:
print("train", Counter(y_train), "test", Counter(y_test))

train Counter({'T2+': 56, 'T1': 54}) test Counter({'T2+': 14, 'T1': 13})


In [37]:
# try predict and evaluate
model = xgbtree["basemodel"]().fit(x_train, y_train)

/labs/gevaertlab/users/yyhhli/miniconda3/envs/lungvae38/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:34:52] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [39]:
y_pred = model.predict(x_test)

In [40]:
from sklearn.metrics import (accuracy_score, average_precision_score, f1_score,
                             make_scorer, mean_absolute_error,
                             mean_absolute_percentage_error,
                             mean_squared_error, precision_score, r2_score,
                             recall_score, roc_auc_score)
CLASSIFICATION_METRIC_DICT = {'Accuracy': accuracy_score,
                              'F1': partial(f1_score, average='weighted'),
                              'Precision': partial(precision_score, average='weighted'),
                              'Recall': partial(recall_score, average='weighted'),
                              'AUROC': partial(roc_auc_score, average='weighted', multi_class='ovr'),
                              'AUPRC': partial(average_precision_score, average='weighted'),
                              }

In [41]:
CLASSIFICATION_METRIC_DICT['F1'](y_test, y_pred)

0.7037037037037037

In [ ]:
# probably it, let's write a .py file about this